In [0]:
!pip install git+https://github.com/raghakot/keras-vis.git -U

In [0]:
from keras.preprocessing import image
from keras import backend as K
import numpy as np

import matplotlib.pyplot as plt

In [0]:
from keras.applications.vgg16 import VGG16
model = VGG16(include_top=True, 
              weights='imagenet', 
              input_tensor=None, 
              input_shape=None)

In [0]:
model.summary() 

In [0]:
img_path = 'cat.jpg'

img = image.load_img(img_path, target_size=(224, 224))
img_tensor = image.img_to_array(img)
img_tensor = np.expand_dims(img_tensor, axis=0)

# 正規化
img_tensor /= 255.

# 形状を表示
print(img_tensor.shape)

In [0]:
plt.imshow(img_tensor[0])
plt.show()

In [0]:
from keras import models

# 上位16層の出力を抽出
layer_outputs = [layer.output for layer in model.layers[2:16]]

# モデル入力を指定すると、これらの出力を返すモデルを作成する。
activation_model = models.Model(inputs=model.input, outputs=layer_outputs)

In [0]:
# 5つのNumpy配列（層の活性化ごとに１つ）のリストを返す
activations = activation_model.predict(img_tensor)

In [0]:
first_layer_activation = activations[0]
print(first_layer_activation.shape)

In [0]:
# 1つ目の層の3番目のチャネルを表示
plt.matshow(first_layer_activation[0, :, :, 3], cmap='viridis')
plt.show()

In [0]:
# 1つ目の層の30番目のチャネルを表示
plt.matshow(first_layer_activation[0, :, :, 30], cmap='viridis')
plt.show()

In [0]:
fourth_layer_activation = activations[3]
print(fourth_layer_activation.shape)

In [0]:
# 4つ目の層の3番目のチャネルを表示
plt.matshow(fourth_layer_activation[0, :, :, 3], cmap='viridis')
plt.show()

In [0]:
# 4つ目の層の3番目のチャネルを表示
plt.matshow(fourth_layer_activation[0, :, :, 30], cmap='viridis')
plt.show()

In [0]:
import keras

# プロットの一部として使用する層の名前
layer_names = []
for layer in model.layers[2:16]:
    layer_names.append(layer.name)

images_per_row = 16

# 特徴マップを表示
for layer_name, layer_activation in zip(layer_names, activations):
    # 特徴マップに含まれている特徴量の数
    n_features = layer_activation.shape[-1]

    # 特徴マップの形状 (1, size, size, n_features)
    size = layer_activation.shape[1]

    # この行列で活性化のチャネルをタイル表示
    n_cols = n_features // images_per_row
    display_grid = np.zeros((size * n_cols, images_per_row * size))

    # 各フィルタを１つの大きな水平グリッドでタイル表示
    for col in range(n_cols):
        for row in range(images_per_row):
            channel_image = layer_activation[0,
                                             :, :,
                                             col * images_per_row + row]
            # 特徴量の見た目をよくするための後処理
            channel_image -= channel_image.mean()
            channel_image /= channel_image.std()
            channel_image *= 64
            channel_image += 128
            channel_image = np.clip(channel_image, 0, 255).astype('uint8')
            display_grid[col * size : (col + 1) * size,
                         row * size : (row + 1) * size] = channel_image

    # グリッド表示
    scale = 1. / size
    plt.figure(figsize=(scale * display_grid.shape[1],
                        scale * display_grid.shape[0]))
    plt.title(layer_name)
    plt.grid(False)
    plt.imshow(display_grid, aspect='auto', cmap='viridis')
    
plt.show()

# CAM（Class Activation Map）

In [0]:
K.clear_session()
model = VGG16(weights='imagenet')

In [0]:
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions
import numpy as np

img_path = '/content/cat.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)

x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

In [0]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [0]:
np.argmax(preds[0])

In [0]:
# 最大の予測ベクトルを取得
model_output = model.output[:, 285]

# VGG16の最後の畳み込み層であるblock5_conv3の出力特徴
last_conv_layer = model.get_layer('block5_conv3')

# block5_conv3の出力特徴マップでの勾配
grads = K.gradients(model_output, last_conv_layer.output)[0]

# 各エントリは特定の特徴マップチャネルの勾配の平均強度
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# pooled_grads と block5_conv3の出力特徴マップの値にアクセスするための関数
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

# これらの2つの値をNumpy配列として取得
pooled_grads_value, conv_layer_output_value = iterate([x])

# 「チャネルの重要度」を特徴マップ配列の各チャネルに掛ける
for i in range(512):
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# 最終的な特徴マップのチャネルごとの平均値がクラスの活性化のヒートマップ
heatmap = np.mean(conv_layer_output_value, axis=-1)

In [0]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

In [0]:
import cv2

# CV2を使って元の画像を読み込む
img = cv2.imread(img_path)

# 元の画像と同じサイズになるようにヒートマップのサイズを変更
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

# ヒートマップをRGBに変換
heatmap = np.uint8(255 * heatmap)

# ヒートマップを元の画像に適用
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# 0.4 は、ヒートマップの強度係数
superimposed_img = heatmap * 0.4 + img

# 保存
cv2.imwrite('/content/cat_heatmap.jpg', superimposed_img)